# PCAP Lab 7 - CUDA String

<h3>Parthivi Choubey</h3>

*180905456*

CSE - B - 6th sem

Roll. no.: 60

# Cuda installation and update

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-60vrlucy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-60vrlucy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=9d671845166a9783110bbed8613e7dde9b7066faf47886dd58caa690017a6853
  Stored in directory: /tmp/pip-ephem-wheel-cache-chbwhn1_/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Question 1

Write a program in CUDA to count the number of times a given word is repeated in a sentence. (Use Atomic function).

In [49]:
%%cu
#include <cuda.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>

__global__ void wordCount(char *str,int *d_count,int *len_words,char *word)
{
    int tempIdx=threadIdx.x;
  int idx=0;
  for(int i=0;i<tempIdx;i++)
    idx += len_words[i]+1;
 
  int flag=0;
  for(int i=0;i<len_words[tempIdx];i++)
  {
      if(str[idx+i]!=word[i])
      {
          flag=1;
          break;
      }
  }
  if(flag==0)
    atomicAdd(d_count,1);
}

int main()
{
    char str[]="hello hello worldworld hello world hello";
  char word[]="hello";
  int num_words=1;
  int count=0;
 
  //Count number of words
  for(int i=0;str[i]!='\0';i++)
    if(str[i]==' ')
      num_words++;
 
  //Count length of each word
  int len_words[num_words];
  int len=0,j=0;
  for(int i=0;str[i]!='\0';i++)
  {
      len++;
      if(str[i]==' ')
      {
          --len;
          len_words[j]=len;
          len=0; j++;
      }
  }
  len_words[j]=len;
 
  // Device copies of the variables
  char *d_str, *d_word;
  int *d_count, *d_len_words;
  
  // Allocate memories to device copies of the objects
  cudaMalloc((void**)&d_str,strlen(str)*sizeof(char));
  cudaMalloc((void**)&d_count,sizeof(int));
  cudaMalloc((void**)&d_word,strlen(word)*sizeof(char));
  cudaMalloc((void**)&d_len_words,num_words*sizeof(int));
 
  //Copy inputs to device
  cudaMemcpy(d_str,str,strlen(str)*sizeof(char),cudaMemcpyHostToDevice);
  cudaMemcpy(d_word,word,strlen(word)*sizeof(char),cudaMemcpyHostToDevice);
  cudaMemcpy(d_len_words,len_words,num_words*sizeof(int),cudaMemcpyHostToDevice);
  
  wordCount<<<1,num_words>>>(d_str,d_count,d_len_words,d_word); // Launch kernel onto the device
  cudaMemcpy(&count,d_count,sizeof(int),cudaMemcpyDeviceToHost); // Copy the result back to the host
  printf("Total occurences of word \"%s\" in \"%s\" is %d\n",word,str,count);
 
  cudaFree(d_str);
  cudaFree(d_count);
  cudaFree(d_word);
  cudaFree(d_len_words);
  return 0;
}

Total occurences of word "hello" in "hello hello worldworld hello world hello" is 4



# Question 2

Write a CUDA program that reads a string Sin and produces the string Sout as follows:

 

                         Input string Sin: PCAP Output string Sout: PCAPPCAPCP

In [50]:
%%cu
#include <cuda.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

__global__ void stringgen(char *Sin, char *Sout)
{
    int len = blockDim.x;
    int tempIdx = threadIdx.x + blockIdx.x * blockDim.x;
    int idx =0;
    for(int i=0;i<tempIdx;i++)
      idx += len-i;
    for(int i=0;i<(len-tempIdx);i++)
      Sout[idx+i] =Sin[i];
}

int main()
{
    char Sin[]="PCAP";
    int SinLen = strlen(Sin);
    int SoutLen = ((SinLen*(SinLen+1))/2);
    char Sout[SoutLen];
    char result[SoutLen];
    char *d_Sin, *d_Sout; // Device copies of the variables
 
    // Allocate memories to device copies of the objects
    cudaMalloc((void**)&d_Sin, SinLen*sizeof(char));
    cudaMalloc((void**)&d_Sout, SoutLen*sizeof(char));
 
    cudaMemcpy(d_Sin,Sin,SinLen*sizeof(char),cudaMemcpyHostToDevice); // Copy input to device
    stringgen<<<1,SinLen>>>(d_Sin,d_Sout); // Launch kernel onto the device
 
    // Copy the result back to the host
    cudaMemcpy(Sout,d_Sout,SoutLen*sizeof(char),cudaMemcpyDeviceToHost);
    Sout[SoutLen] = '\0';
    printf("Final String is: %s\n",Sout);
 
    cudaFree(d_Sin);
    cudaFree(d_Sout);
    return 0;
}

Final String is: PCAPPCAPCP

